# Lab 04: LSTM yourself - solution

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5242_2025_codes/labs_lecture06/lab04_lstm_yourself'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 274 sec ( 4.56 min)<br> 
* Time for 1 epoch on GPU : 10.1 sec w/ GeForce GTX 1080 Ti <br>

In [2]:
device= torch.device("cuda")
# device= torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))

cuda
cuda available with GPU: NVIDIA RTX A5000


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [3]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [4]:
bs = 20

vocab_size = 10000


### Make a LTSM class

Implement the vanilla RNN network without PyTorch:

$$
\begin{aligned}
&\tilde{h}_t = \tanh ( Rh_{t-1} + V g_t) \\
&c_t = \theta_t \odot c_{t-1} + \eta_t \odot \tilde{h}_t \\
&h_t = \psi_t \odot \tanh (c_t) \\
&\textrm{with}\\
&\theta_t = \textrm{sigmoid}(Ah_{t-1}+Bg_t)\\
&\eta_t = \textrm{sigmoid}(Ch_{t-1}+Dg_t)\\
&\psi_t = \textrm{sigmoid}(Eh_{t-1}+Fg_t)\\
\end{aligned}
$$



In [13]:
class my_LSTM(nn.Module):
    
    def __init__(self, hidden_size):
        super().__init__()
        self.R = nn.Linear( hidden_size , hidden_size )
        self.V = nn.Linear( hidden_size , hidden_size )
        self.A = nn.Linear( hidden_size , hidden_size )
        self.B = nn.Linear( hidden_size , hidden_size )
        self.C = nn.Linear( hidden_size , hidden_size )
        self.D = nn.Linear( hidden_size , hidden_size )
        self.E = nn.Linear( hidden_size , hidden_size )
        self.F = nn.Linear( hidden_size , hidden_size )
        
    def forward(self, g_seq , hc_init ):
        h_init, c_init = hc_init
        h_t_pre = h_init
        c_t_pre = c_init
        V_g_all_t = self.V(g_seq) 
        B_g_all_t = self.B(g_seq) 
        D_g_all_t = self.D(g_seq) 
        F_g_all_t = self.F(g_seq) 
        h_seq = []
        c_seq = []
        num_t = g_seq.size(0)
        for t in range(num_t):
            R_h_t_pre = self.R(h_t_pre) 
            A_h_t_pre = self.A(h_t_pre) 
            C_h_t_pre = self.C(h_t_pre) 
            E_h_t_pre = self.E(h_t_pre) 
            theta_t = torch.sigmoid( A_h_t_pre + B_g_all_t[t] )
            eta_t   = torch.sigmoid( C_h_t_pre + D_g_all_t[t] )
            psi_t   = torch.sigmoid( E_h_t_pre + F_g_all_t[t] )
            h_tilde_t = torch.tanh( R_h_t_pre + V_g_all_t[t] )
            c_t = theta_t * c_t_pre + eta_t * h_tilde_t
            h_t = psi_t * torch.tanh( c_t )
            h_seq.append(h_t)
            c_seq.append(c_t)
            h_t_pre = h_t
            c_t_pre = c_t
        h_seq = torch.cat(h_seq) 
        h_final = h_seq[-1,:,:].unsqueeze(0) 
        c_final = c_t
        return h_seq , (h_final,c_final)

    
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = my_LSTM( hidden_size )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init, c_init ):
        
        g_seq                      =   self.layer1( word_seq )  
        h_seq , (h_final,c_final)  =   self.layer2( g_seq , (h_init,c_init) )     
        score_seq                  =   self.layer3( h_seq )
        
        return score_seq,  h_final , c_final


### Build the net. Choose the hidden size to be 300. How many parameters in total?

In [15]:
hidden_size=300

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 300)
  (layer2): my_LSTM(
    (R): Linear(in_features=300, out_features=300, bias=True)
    (V): Linear(in_features=300, out_features=300, bias=True)
    (A): Linear(in_features=300, out_features=300, bias=True)
    (B): Linear(in_features=300, out_features=300, bias=True)
    (C): Linear(in_features=300, out_features=300, bias=True)
    (D): Linear(in_features=300, out_features=300, bias=True)
    (E): Linear(in_features=300, out_features=300, bias=True)
    (F): Linear(in_features=300, out_features=300, bias=True)
  )
  (layer3): Linear(in_features=300, out_features=10000, bias=True)
)
There are 6732400 (6.73 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [16]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [17]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 5
* sequence length = 35

In [18]:
criterion = nn.CrossEntropyLoss()

my_lr = 5

seq_length = 35

### Function to evaluate the network on the test set

In [19]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)
   
    h=h.to(device)
    c=c.to(device)
       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h, c  = net( minibatch_data, h , c)
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
        c=c.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 8 passes through the training set.

In [21]:
start=time.time()

for epoch in range(8):
    
    # divide the learning rate by 3 except after the first epoch
    if epoch >= 2:
        my_lr = my_lr / 3
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h and c to be the zero vector
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)

    # send them to the gpu    
    h=h.to(device)
    c=c.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        c=c.detach()
        h=h.requires_grad_()
        c=c.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h, c  = net( minibatch_data, h , c)
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()   
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 72.81249976158142 	 lr= 5 	 exp(loss)= 277.84873897015564
test: exp(loss) =  175.27791294836064

epoch= 1 	 time= 146.63453888893127 	 lr= 5 	 exp(loss)= 126.55364295431386
test: exp(loss) =  133.37028600196277

epoch= 2 	 time= 220.80677247047424 	 lr= 1.6666666666666667 	 exp(loss)= 80.47941602868258
test: exp(loss) =  113.88088006579213

epoch= 3 	 time= 291.6459333896637 	 lr= 0.5555555555555556 	 exp(loss)= 66.30208510205978
test: exp(loss) =  109.75163343720969

epoch= 4 	 time= 359.571711063385 	 lr= 0.1851851851851852 	 exp(loss)= 61.46460867290013
test: exp(loss) =  108.32787895110684

epoch= 5 	 time= 427.32110476493835 	 lr= 0.0617283950617284 	 exp(loss)= 59.724057657481985
test: exp(loss) =  107.55765690090652

epoch= 6 	 time= 495.0989077091217 	 lr= 0.0205761316872428 	 exp(loss)= 59.09803784937284
test: exp(loss) =  107.05129669700894

epoch= 7 	 time= 562.9837806224823 	 lr= 0.006858710562414266 	 exp(loss)= 58.870344887407285
test: exp(loss) =  106.7

### Choose one sentence (taken from the test set)

In [23]:
# SENTENCES FROM TEST SET

sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence2 = "some analysts expect oil prices to remain relatively"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# OR MAKE YOUR OWN SENTENCE. 
# NO CAPITAL LETTER ALLOWED. EACH WORD MUST BE IN THE ALLOWED VOCABULARY OF 10,000 WORDS

sentence6= "he was very"


# CHOOSE THE SENTENCE
mysentence = sentence3

### Convert the sentence into a vector, then send to GPU

In [24]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[1786],
        [8705],
        [3246],
        [ 416],
        [  27],
        [  35],
        [2664],
        [ 209],
        [ 108],
        [  32],
        [3017]], device='cuda:0')


### Set the initial hidden state to zero, then run the LSTM.

In [25]:
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
h=h.to(device)
c=c.to(device)

scores , h, c = net(minibatch_data , h, c)

### Display the network prediction for the next word

In [26]:
print(mysentence, '... \n')

utils.show_next_word(scores)

prices averaging roughly $ N a barrel higher in the third ... 

94.8%	 quarter
0.5%	 and
0.4%	 week
0.4%	 period
0.4%	 year
0.4%	 month
0.3%	 world
0.3%	 half
0.3%	 consecutive
0.2%	 <eos>
0.1%	 of
0.1%	 game
0.1%	 market
0.1%	 account
0.1%	 area
0.1%	 range
0.1%	 fiscal
0.1%	 hour
0.1%	 or
0.0%	 level
0.0%	 floor
0.0%	 1970s
0.0%	 term
0.0%	 season
0.0%	 day
0.0%	 1980s
0.0%	 side
0.0%	 <unk>
0.0%	 quarters
0.0%	 third
